#### Aggregates ZCTA-level case data to the PUMA level by merging with ZCTA-to-PUMA crosswalk and grouping by PUMA.

In [1]:
import pandas as pd
df = pd.read_csv("tests-by-zcta-04-29.csv")
df.rename(columns = {"MODZCTA":"zcta10"}, inplace = True)
df_zp = pd.read_excel("nyc_zcta10_to_puma10.xls", sheet_name = 1)[['zcta10','puma10','pumaname']]
dfp = pd.merge(df, df_zp)

In [2]:
## Extracts PUMA names from the sorted dataset, to be appended after groupby.
names = dfp.sort_values('puma10')['pumaname']
names.drop_duplicates(inplace = True)
names = names.reset_index()['pumaname']

In [3]:
dfp_agg = dfp.groupby('puma10').sum()[['Positive', 'Total']]
dfp_agg.reset_index(inplace = True)
dfp_agg['pumaname'] = names

In [4]:
dfp_agg[:3]

,puma10,Positive,Total,pumaname
0,3701,1991,4685,"NYC-Bronx Community District 8--Riverdale, Fie..."
1,3702,2390,4856,"NYC-Bronx Community District 12--Wakefield, Wi..."
2,3703,4456,10329,"NYC-Bronx Community District 10--Co-op City, P..."


#### Maps each PUMA to the ID of their approximate Community District. For the 4 PUMAs that comprise 2 CDs each, I mapped to the first ID of the pair e.g. MN01 for MN01-MN02, MN04 for MN04-MN05, BX01 for BX01-BX02, and BX03 for BX03-BX06.

In [5]:
puma_to_id = pd.read_csv('puma_to_id.csv')
df1 = pd.merge(dfp_agg, puma_to_id)

In [6]:
## Observe that PUMA 3705 containing BX03 and BX06 is assigned to 203.
df1[:5]

,puma10,Positive,Total,pumaname,ID
0,3701,1991,4685,"NYC-Bronx Community District 8--Riverdale, Fie...",208
1,3702,2390,4856,"NYC-Bronx Community District 12--Wakefield, Wi...",212
2,3703,4456,10329,"NYC-Bronx Community District 10--Co-op City, P...",210
3,3704,2455,4706,NYC-Bronx Community District 11--Pelham Parkwa...,211
4,3705,3285,6571,"NYC-Bronx Community District 3 & 6--Belmont, C...",203


#### Merges COVID case counts with CHP dataset containing demographic and socioeconomic indicators.

In [7]:
chp = pd.read_excel("2018-chp-dataset.xlsx", sheet_name = "CHP_all_data")
chp = chp[:59]

In [8]:
chp[:3]

,ID,Borough,Name,under18_rate,over65_rate,pct_asian_nh,pct_black_nh,pct_hispanic,pct_white_nh,pct_foreign_born,...,upper_95CL.9,Obesity,lower_95CL.10,upper_95CL.10,Diabetes,lower_95CL.11,upper_95CL.11,Hypertension,lower_95CL.12,upper_95CL.12
0,101,Manhattan,Financial District,12.8,12.2,15.7,2.3,6.4,72.0,23.9,...,53.9,4.30,2.4,6.2,3.95,1.9,6.0,15.80,11.7,19.9
1,102,Manhattan,Greenwich Village and Soho,12.8,12.2,15.7,2.3,6.4,72.0,23.9,...,53.9,4.30,2.4,6.2,3.95,1.9,6.0,15.80,11.7,19.9
2,103,Manhattan,Lower East Side and Chinatown,11.7,17.0,32.1,7.4,25.1,32.7,34.9,...,53.4,10.45,6.9,14.0,11.25,7.7,14.8,22.55,17.9,27.2


In [9]:
dfc = pd.merge(df1, chp)
dfc.columns

Index(['puma10', 'Positive', 'Total', 'pumaname', 'ID', 'Borough', 'Name',
       'under18_rate', 'over65_rate', 'pct_asian_nh', 'pct_black_nh',
       'pct_hispanic', 'pct_white_nh', 'pct_foreign_born', 'poverty_rate',
       'count_hosp_clinic', 'below150poverty', 'MgBSciArtP', 'SrvcP',
       'SalesOffP', 'NRCnstMntP', 'PrdTrnsMMP', 'MdHHIncE',
       'Median household income', 'Overall_Pop', 'Ltd_Eng_Prof', 'lower_95CL',
       'upper_95CL', 'Edu_Did_Not_Complete_HS', 'lower_95CL.1', 'upper_95CL.1',
       'Edu_HSGrad_Some_College', 'lower_95CL.2', 'upper_95CL.2',
       'Edu_College_Degree_And_Higher', 'lower_95CL.3', 'upper_95CL.3',
       'Unemployment', 'lower_95CL.4', 'upper_95CL.4', 'Rent_Burden',
       'lower_95CL.5', 'upper_95CL.5', 'Uninsured',
       'Uninsured_reliability_note', 'lower_95CL.6', 'upper_95CL.6',
       'Unmet_Med_Care', 'Unmet_Med_Care_reliability_note', 'lower_95CL.7',
       'upper_95CL.7', 'Avoidable_Adult_Hosp', 'lower_95CL.8', 'upper_95CL.8',
       

In [10]:
dfc['COVIDCasesPer10k'] = (dfc['Positive'] / dfc['Overall_Pop'] * 10000)

In [11]:
dfc['TestsPer10k'] = (dfc['Total']/dfc['Overall_Pop'] * 10000)

In [12]:
reg_vars =      ['puma10',
                 'Borough',
                 'COVIDCasesPer10k',
                 'TestsPer10k',
                 'Overall_Pop',
                 'pct_asian_nh', 'pct_black_nh', 'pct_hispanic', 'pct_white_nh', 
                 'pct_foreign_born','Ltd_Eng_Prof',
                 'under18_rate', 'over65_rate',
                 'poverty_rate', 'below150poverty', 'Unemployment', 'Rent_Burden',
                 'MgBSciArtP', 'SrvcP', 'SalesOffP', 'NRCnstMntP', 'PrdTrnsMMP', 
                 'MdHHIncE', 'Median household income', 
                 'Edu_Did_Not_Complete_HS', 'Edu_HSGrad_Some_College', 'Edu_College_Degree_And_Higher',
                 'HPV_Vaccination_All','Diabetes','Obesity','Hypertension','count_hosp_clinic',]
df_export = dfc[reg_vars]

In [13]:
df_export.to_excel("covid_reg_dataset.xlsx")